<a href="https://colab.research.google.com/github/tomasrojas88/Data_Science_1/blob/main/Entrega_Final_Data_Science_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1) Introducción

En el presente trabajo se analiza un conjunto de datos de satisfacción de pasajeros con el objetivo de aplicar técnicas de Machine Learning supervisado a un problema real de negocio. El análisis se orienta a comprender qué factores del servicio influyen en la percepción de satisfacción de los clientes y a desarrollar modelos capaces de predecir si un pasajero se encuentra satisfecho o no.

La satisfacción del cliente constituye un indicador clave para las aerolíneas, dado que impacta directamente en la fidelización, la reputación de la marca y la toma de decisiones estratégicas vinculadas a la calidad del servicio. Contar con modelos predictivos permite no solo anticipar comportamientos de los clientes, sino también identificar oportunidades concretas de mejora basadas en datos.

Para el desarrollo del trabajo se utiliza un conjunto de datos de encuestas de satisfacción disponible en la plataforma Kaggle. El enfoque adoptado es práctico y aplicado, recorriendo las distintas etapas de un proyecto de Data Science, desde el análisis exploratorio y la preparación de los datos, hasta el entrenamiento, evaluación y selección del modelo de clasificación con mejor desempeño.

# 2) Problema a resolver

El objetivo principal de este trabajo es abordar el problema como una tarea de clasificación supervisada, cuyo propósito es predecir si un pasajero se encuentra satisfecho o no con el servicio recibido. A partir de las variables disponibles en el conjunto de datos, se busca entrenar modelos de Machine Learning capaces de realizar esta predicción de manera precisa.

La variable objetivo (target) corresponde al nivel de satisfacción del pasajero, mientras que el resto de las variables del dataset se utilizan como variables explicativas, representando distintas características del servicio, del cliente y de la experiencia de viaje.

El interés del análisis no se limita únicamente a la obtención de una predicción, sino también a comprender qué aspectos del servicio tienen mayor influencia en la satisfacción de los clientes, aportando información relevante para la toma de decisiones y la mejora de la experiencia del pasajero.

# 3) Carga de datos

En esta etapa se realiza la carga del conjunto de datos y una primera inspección general de su estructura. El objetivo es verificar que los datos hayan sido importados correctamente y comprender los principales características del dataset antes de avanzar con el análisis.

Se analiza la cantidad de registros y columnas disponibles, los tipos de variables presentes y la existencia de posibles valores faltantes. Esta revisión inicial permite detectar de forma temprana eventuales problemas de calidad de datos y dimensionar el alcance de la información disponible.

Contar con esta visión general del dataset resulta fundamental para orientar las etapas posteriores del proyecto, en particular el análisis exploratorio, la ingeniería de atributos y el entrenamiento de los modelos de Machine Learning.

In [97]:
# Importar librerías
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import warnings
from xgboost import XGBClassifier

In [98]:
# Configuraciones opcionales para mejor visualización
sns.set_theme(style="whitegrid") # Establece un tema estético para los gráficos
plt.rcParams['figure.figsize'] = (12, 6) # Establece un tamaño de figura por defecto
warnings.filterwarnings('ignore') # Ignora advertencias (útil para presentaciones)

In [99]:
# Descarga del archivo CSV desde un repositorio remoto y verificación de su estructura

import requests

url = "https://github.com/tomasrojas88/Data_Science_1/raw/refs/heads/main/Dataset%20Proyecto%20Coderhouse%20Satisfacci%C3%B3n%20Clientes%20en%20Aerolineas2.csv"

response = requests.get(url)  # Hace una solicitud GET para obtener el contenido del archivo
print(response.text[:300])  # Imprime los primeros 300 caracteres del archivo para ver cómo está delimitado
response = requests.get(url)  # Hace una solicitud GET para obtener el contenido del archivo
print(response.text[:300])  # Imprime los primeros 300 caracteres del archivo para ver cómo está delimitado

id_Encuesta;Identificación;Género;Tipo de Cliente;Edad;Tipo de Viaje;Clase;Distancia de Vuelo;Servicio de wifi a bordo;Hora de salida/llegada conveniente;Facilidad de reserva en línea;Ubicación de la puerta;Comida y bebida;Embarque en línea;Comodidad del asiento;Entretenimiento a bordo;Servicio a bo
id_Encuesta;Identificación;Género;Tipo de Cliente;Edad;Tipo de Viaje;Clase;Distancia de Vuelo;Servicio de wifi a bordo;Hora de salida/llegada conveniente;Facilidad de reserva en línea;Ubicación de la puerta;Comida y bebida;Embarque en línea;Comodidad del asiento;Entretenimiento a bordo;Servicio a bo


In [100]:
# Carga del dataset en un DataFrame de pandas y exploración inicial de su estructura

df = pd.read_csv(url, sep=';', encoding='latin1')

print("Tamaño del dataset (filas, columnas):", df.shape)  # Para saber cuántos registros tenemos

df.info()  # Muestra tipos de datos y cantidad de valores no nulos por columna
df.describe()  # Estadísticos descriptivos básicos de las columnas numéricas

Tamaño del dataset (filas, columnas): (25976, 25)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25976 entries, 0 to 25975
Data columns (total 25 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   id_Encuesta                           25976 non-null  int64  
 1   Identificación                        25976 non-null  int64  
 2   Género                                25976 non-null  object 
 3   Tipo de Cliente                       25976 non-null  object 
 4   Edad                                  25976 non-null  int64  
 5   Tipo de Viaje                         25976 non-null  object 
 6   Clase                                 25976 non-null  object 
 7   Distancia de Vuelo                    25976 non-null  int64  
 8   Servicio de wifi a bordo              25976 non-null  int64  
 9   Hora de salida/llegada conveniente    25976 non-null  int64  
 10  Facilidad de reserva en línea   

,id_Encuesta,Identificación,Edad,Distancia de Vuelo,Servicio de wifi a bordo,Hora de salida/llegada conveniente,Facilidad de reserva en línea,Ubicación de la puerta,Comida y bebida,Embarque en línea,Comodidad del asiento,Entretenimiento a bordo,Servicio a bordo,Servicio de espacio para las piernas,Manejo de equipaje,Servicio de facturación,Servicio a bordo.1,Limpieza,Retraso en la salida en minutos,Retraso en la llegada en minutos
count,25976.000000,25976.000000,25976.000000,25976.000000,25976.000000,25976.000000,25976.000000,25976.000000,25976.000000,25976.000000,25976.000000,25976.000000,25976.000000,25976.000000,25976.000000,25976.000000,25976.000000,25976.000000,25976.00000,25893.000000
mean,12987.500000,65005.657992,39.620958,1193.788459,2.724746,3.046812,2.756775,2.977094,3.215353,3.261665,3.449222,3.357753,3.385664,3.350169,3.633238,3.314175,3.649253,3.286226,14.30609,14.740857
std,7498.769632,37611.526647,15.135685,998.683999,1.335384,1.533371,1.412951,1.282133,1.331506,1.355536,1.320090,1.338299,1.282088,1.318862,1.176525,1.269332,1.180681,1.319330,37.42316,37.517539
min,0.000000,17.000000,7.000000,31.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.00000,0.000000
25%,6493.750000,32170.500000,27.000000,414.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,3.000000,3.000000,3.000000,2.000000,0.00000,0.000000
50%,12987.500000,65319.500000,40.000000,849.000000,3.000000,3.000000,3.000000,3.000000,3.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,3.000000,4.000000,3.000000,0.00000,0.000000
75%,19481.250000,97584.250000,51.000000,1744.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,5.000000,4.000000,4.000000,4.000000,5.000000,4.000000,5.000000,4.000000,12.00000,13.000000
max,25975.000000,129877.000000,85.000000,4983.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,1128.00000,1115.000000


# 4) EDA (Exploración y limpieza)

En esta sección se lleva a cabo un análisis exploratorio de los datos (EDA) con el objetivo de comprender la distribución de las variables, identificar la presencia de valores faltantes y detectar posibles inconsistencias u observaciones atípicas. Este análisis inicial permite obtener una primera comprensión del comportamiento de los datos y su calidad.

Durante esta etapa también se realizan tareas básicas de limpieza y validación, asegurando que el conjunto de datos se encuentre en condiciones adecuadas para su posterior utilización en los modelos de Machine Learning.

El EDA resulta fundamental para detectar patrones relevantes y validar supuestos iniciales sobre la relación entre las variables explicativas y la satisfacción del pasajero, sirviendo como base para las etapas de ingeniería de atributos y modelado.

In [101]:
# Conteo de valores nulos por columna
df.isnull().sum()  # Nos permite detectar columnas con datos faltantes


,0
id_Encuesta,0
Identificación,0
Género,0
Tipo de Cliente,0
Edad,0
Tipo de Viaje,0
Clase,0
Distancia de Vuelo,0
Servicio de wifi a bordo,0
Hora de salida/llegada conveniente,0


In [102]:
# Limpieza de la columna 'Retraso en la llegada en minutos'

# Convertimos a numérico forzando errores a NaN
df["Retraso en la llegada en minutos"] = pd.to_numeric(
    df["Retraso en la llegada en minutos"], errors="coerce"
)  # 'errors=coerce' convierte textos inválidos en NaN

# Contamos cuántos NaN quedaron después de la conversión
df["Retraso en la llegada en minutos"].isna().sum()  # Para ver cuántos valores quedaron como faltantes

# Reemplazamos NaN por 0 (criterio: valor faltante = sin retraso registrado)
df["Retraso en la llegada en minutos"].fillna(0, inplace=True)  # Imputamos con 0

# Convertimos a entero para que quede un tipo consistente
df["Retraso en la llegada en minutos"] = df["Retraso en la llegada en minutos"].astype("int64")  # Cast a entero


In [103]:
# Búsqueda y eliminación de filas duplicadas
duplicados = df.duplicated().sum()  # Cuántas filas están repetidas completamente
print("Filas duplicadas:", duplicados)

df = df.drop_duplicates()  # Eliminamos duplicados si existieran
print("Nuevo tamaño del dataset:", df.shape)  # Verificamos el nuevo tamaño


Filas duplicadas: 0
Nuevo tamaño del dataset: (25976, 25)


En esta etapa se realizaron tareas puntuales de limpieza de datos para asegurar la calidad y consistencia del dataset antes del modelado. Por un lado, se trabajó sobre la variable “Retraso en la llegada en minutos”, convirtiéndola a formato numérico y tratando los valores faltantes, con el objetivo de evitar errores y mantener una representación coherente de la información. Por otro lado, se verificó la existencia de registros duplicados en el conjunto de datos, eliminándolos en caso de ser necesario, para evitar sesgos en el entrenamiento de los modelos

# 5) Feature Engineering (preparación de variables)

En esta etapa se realiza la preparación de las variables con el objetivo de adecuar el conjunto de datos para su utilización en modelos de Machine Learning. Este proceso incluye la transformación de la variable objetivo y la codificación de las variables explicativas, garantizando su compatibilidad con los algoritmos empleados.

En particular, la variable objetivo es transformada en un formato binario para abordar el problema como una tarea de clasificación supervisada. Asimismo, las variables categóricas son codificadas mediante One-Hot Encoding, permitiendo su utilización en modelos que requieren variables numéricas, como XGBoost.

Estas transformaciones buscan mejorar la capacidad predictiva de los modelos y evitar problemas derivados de escalas, formatos incompatibles o representaciones no numéricas de los datos.

In [104]:
df['Satisfacción'] = df['Satisfacción'].map({'neutral o insatisfecho/a': 0, 'satisfecho/a': 1})

In [105]:
# Convertir variables categóricas en variables dummy para que XGBoost pueda entrenar
X_encoded = pd.get_dummies(X, drop_first=True)


In [106]:
# Dividir el dataset ya codificado en entrenamiento y test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


In [107]:
# Verificar que no quedaron columnas tipo object
X_train.dtypes.value_counts()


,count
int64,20
bool,5


En esta etapa se realizó la preparación final de las variables para el modelado. En primer lugar, la variable objetivo fue transformada a un formato binario, permitiendo abordar el problema como una tarea de clasificación supervisada. Luego, las variables categóricas fueron codificadas mediante One-Hot Encoding, de modo de convertirlas en variables numéricas compatibles con los modelos a utilizar, en particular XGBoost. Finalmente, se dividió el conjunto de datos ya codificado en entrenamiento y prueba, verificando que no quedaran variables de tipo texto, lo que garantiza la correcta ejecución de los modelos de Machine Learning.

# 6) Split Train/Test

En esta etapa el conjunto de datos se divide en conjuntos de entrenamiento y prueba, con el objetivo de evaluar el desempeño de los modelos sobre datos no vistos durante el proceso de entrenamiento.

Esta separación resulta fundamental para medir la capacidad de generalización de los modelos y evitar conclusiones engañosas basadas únicamente en su desempeño sobre los datos de entrenamiento. Para asegurar la comparabilidad entre los distintos algoritmos evaluados, se utiliza un único esquema de partición que se mantiene constante a lo largo de todo el proceso de modelado.

Además, se aplica una división estratificada, preservando la proporción de clases de la variable objetivo en ambos subconjuntos.

In [108]:
# Separación de variables predictoras (X) y variable objetivo (y)
X = df.drop('Satisfacción', axis=1)
y = df['Satisfacción']

# División del dataset en conjuntos de entrenamiento y prueba
# Se mantiene la proporción de clases mediante stratify
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# Chequeo rápido de tamaños para validar el split
print(X_train.shape, X_test.shape)


(20780, 24) (5196, 24)


# 7) Modelos

Se entrenaron distintos modelos de clasificación con el objetivo de comparar enfoques y evaluar cuál se adapta mejor al problema planteado.

Cada modelo fue entrenado bajo las mismas condiciones iniciales, permitiendo una comparación justa en términos de desempeño, simplicidad y capacidad predictiva.

In [109]:
# Entrena un modelo de Regresión Logística como baseline (con variables categóricas codificadas)
from sklearn.linear_model import LogisticRegression
import pandas as pd

# Codifica variables categóricas para que el modelo pueda trabajar solo con números
X_train_encoded = pd.get_dummies(X_train, drop_first=True)
X_test_encoded = pd.get_dummies(X_test, drop_first=True)

# Alinea columnas entre train y test para evitar desajustes
X_test_encoded = X_test_encoded.reindex(columns=X_train_encoded.columns, fill_value=0)

# Define y entrena el modelo de Regresión Logística
modelo_logistico = LogisticRegression(max_iter=1000, random_state=42)
modelo_logistico.fit(X_train_encoded, y_train)


LogisticRegression(max_iter=1000, random_state=42)

In [110]:
# Evalúa el modelo logístico con métricas básicas
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Predicciones sobre el conjunto de prueba
y_pred = modelo_logistico.predict(X_test_encoded)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Reporte de clasificación
print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred))

# Matriz de confusión
print("\nMatriz de confusión:")
print(confusion_matrix(y_test, y_pred))


Accuracy: 0.7698229407236336

Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.80      0.78      0.79      2915
           1       0.73      0.75      0.74      2281

    accuracy                           0.77      5196
   macro avg       0.77      0.77      0.77      5196
weighted avg       0.77      0.77      0.77      5196


Matriz de confusión:
[[2284  631]
 [ 565 1716]]


In [111]:
# Entrena un modelo Random Forest como segundo modelo (ensemble)
from sklearn.ensemble import RandomForestClassifier

modelo_rf = RandomForestClassifier(
    n_estimators=200,
    random_state=42,
    n_jobs=-1
)

modelo_rf.fit(X_train_encoded, y_train)


RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=42)

In [112]:
# Evalúa el modelo Random Forest con métricas básicas
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Predicciones sobre el conjunto de prueba
y_pred_rf = modelo_rf.predict(X_test_encoded)

# Accuracy
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print("Accuracy Random Forest:", accuracy_rf)

# Reporte de clasificación
print("\nReporte de clasificación Random Forest:")
print(classification_report(y_test, y_pred_rf))

# Matriz de confusión
print("\nMatriz de confusión Random Forest:")
print(confusion_matrix(y_test, y_pred_rf))


Accuracy Random Forest: 0.9563125481139338

Reporte de clasificación Random Forest:
              precision    recall  f1-score   support

           0       0.95      0.97      0.96      2915
           1       0.96      0.94      0.95      2281

    accuracy                           0.96      5196
   macro avg       0.96      0.95      0.96      5196
weighted avg       0.96      0.96      0.96      5196


Matriz de confusión Random Forest:
[[2821   94]
 [ 133 2148]]


In [113]:
print(f"Regresión logística:\nAccuracy = {accuracy:.4f}")
print(f"Random Forest:\nAccuracy = {accuracy_rf:.4f}")

Regresión logística:
Accuracy = 0.7698
Random Forest:
Accuracy = 0.9563


In [114]:
# Definir el modelo XGBoost para clasificación binaria
from xgboost import XGBClassifier

xgb_model = XGBClassifier(
    objective='binary:logistic',  # Clasificación binaria
    eval_metric='logloss',        # Métrica interna de entrenamiento
    random_state=42,              # Reproducibilidad
    use_label_encoder=False       # Evita warnings innecesarios
)

# Entrenar el modelo con el conjunto de entrenamiento
xgb_model.fit(X_train_encoded, y_train)

# Generar predicciones de clase sobre el conjunto de prueba
y_pred_xgb = xgb_model.predict(X_test_encoded)

# Obtener probabilidades para la clase positiva (necesarias para ROC-AUC)
y_proba_xgb = xgb_model.predict_proba(X_test_encoded)[:, 1]


In [115]:
# Generar predicciones de clase sobre el conjunto de prueba
# Se utilizan los datos codificados, consistentes con el entrenamiento del modelo
y_pred_xgb = xgb_model.predict(X_test_encoded)

# Obtener probabilidades para la clase positiva (necesarias para calcular ROC-AUC)
y_proba_xgb = xgb_model.predict_proba(X_test_encoded)[:, 1]

# Importar métricas de evaluación
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix

# Calcular Accuracy del modelo XGBoost
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)

# Calcular ROC-AUC del modelo XGBoost
roc_auc_xgb = roc_auc_score(y_test, y_proba_xgb)

# Mostrar métricas principales
print(f"Accuracy XGBoost: {accuracy_xgb:.4f}")
print(f"ROC-AUC XGBoost: {roc_auc_xgb:.4f}")

# Mostrar reporte de clasificación completo
print("\nReporte de clasificación XGBoost:")
print(classification_report(y_test, y_pred_xgb))

# Mostrar matriz de confusión
print("\nMatriz de confusión XGBoost:")
print(confusion_matrix(y_test, y_pred_xgb))


Accuracy XGBoost: 0.9580
ROC-AUC XGBoost: 0.9944

Reporte de clasificación XGBoost:
              precision    recall  f1-score   support

           0       0.96      0.97      0.96      2915
           1       0.96      0.95      0.95      2281

    accuracy                           0.96      5196
   macro avg       0.96      0.96      0.96      5196
weighted avg       0.96      0.96      0.96      5196


Matriz de confusión XGBoost:
[[2816   99]
 [ 119 2162]]


A partir del entrenamiento y evaluación de los distintos modelos de clasificación, se observa una clara diferencia en el desempeño al comparar enfoques de distinta complejidad. La Regresión Logística, utilizada como modelo base, presenta un rendimiento aceptable y permite establecer una referencia inicial, aunque muestra limitaciones para capturar relaciones más complejas entre las variables. Si bien su interpretación es sencilla, su capacidad predictiva resulta inferior en comparación con los modelos más avanzados.

El modelo de Random Forest evidencia una mejora significativa en todas las métricas evaluadas, logrando un alto nivel de accuracy y un balance sólido entre precisión y recall para ambas clases. Esto sugiere una mayor capacidad para modelar interacciones no lineales y reducir errores de clasificación, posicionándolo como una alternativa robusta frente al modelo base.

Finalmente, XGBoost se destaca como el modelo con mejor desempeño general. Presenta los valores más altos tanto en accuracy como en ROC-AUC, lo que indica una excelente capacidad de discriminación entre pasajeros satisfechos y no satisfechos. Los resultados obtenidos reflejan una mayor eficiencia en la generalización sobre datos no vistos, consolidando a XGBoost como el modelo más adecuado para el problema planteado.

En conjunto, el análisis confirma que, a medida que se incorporan modelos de mayor complejidad y capacidad de aprendizaje, se logra una mejora sustancial en el desempeño predictivo. En función de los resultados obtenidos, XGBoost se selecciona como el modelo final recomendado para abordar la predicción de la satisfacción de los pasajeros.

# 8) Validación cruzada

Para obtener una evaluación más robusta de los modelos, se aplicó validación cruzada.
Este enfoque permite reducir la dependencia de una única partición de los datos y obtener métricas más estables y confiables.

Los resultados de la validación cruzada ayudaron a identificar qué modelos mantienen un buen desempeño de forma consistente.

In [116]:
# Evalúa los modelos usando validación cruzada con AUC como métrica
from sklearn.model_selection import StratifiedKFold, cross_val_score

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Validación cruzada para Regresión Logística
auc_logistico = cross_val_score(
    modelo_logistico,
    X_train_encoded,
    y_train,
    cv=cv,
    scoring="roc_auc"
)

# Validación cruzada para Random Forest
auc_rf = cross_val_score(
    modelo_rf,
    X_train_encoded,
    y_train,
    cv=cv,
    scoring="roc_auc"
)

# Validación cruzada para XGBoost
auc_xgb = cross_val_score(
    xgb_model,
    X_train_encoded,
    y_train,
    cv=cv,
    scoring="roc_auc"
)

print("AUC promedio XGBoost:", auc_xgb.mean())


print("AUC promedio Regresión Logística:", auc_logistico.mean())
print("AUC promedio Random Forest:", auc_rf.mean())


AUC promedio XGBoost: 0.9933238563976335
AUC promedio Regresión Logística: 0.864229139131918
AUC promedio Random Forest: 0.9911338629603661


La aplicación de validación cruzada permitió obtener una evaluación más robusta del desempeño de los modelos, reduciendo la dependencia de una única partición de los datos. Los resultados confirman las conclusiones obtenidas en la evaluación sobre el conjunto de prueba, mostrando que la Regresión Logística presenta un desempeño inferior en términos de capacidad de discriminación.

Por el contrario, los modelos de Random Forest y XGBoost mantienen valores elevados de ROC-AUC de forma consistente a lo largo de los distintos folds, lo que indica una buena capacidad de generalización. Entre ellos, XGBoost presenta el mejor desempeño promedio, consolidándose como el modelo más robusto y estable para el problema planteado.

# 9) Optimización de hiperparámetros (GridSearch / RandomizedSearch)

En esta etapa se optimizaron los hiperparámetros del modelo Random Forest utilizando técnicas de búsqueda sistemática.

El objetivo fue mejorar el rendimiento de los modelos ajustando sus parámetros internos, logrando un equilibrio entre precisión y capacidad de generalización.

Dado que la Regresión Logística fue utilizada como modelo base y que XGBoost ya presentaba un desempeño elevado y estable, la optimización de hiperparámetros se focalizó en el modelo de Random Forest, con el objetivo de evaluar su sensibilidad a la configuración de parámetros y reforzar su capacidad de generalización.

In [117]:
# Optimiza hiperparámetros del Random Forest con una grilla reducida
from sklearn.model_selection import GridSearchCV

param_grid = {
    "n_estimators": [100],
    "max_depth": [None, 10],
    "min_samples_split": [2],
    "min_samples_leaf": [1]
}

grid_rf = GridSearchCV(
    estimator=modelo_rf,
    param_grid=param_grid,
    cv=3,
    scoring="roc_auc",
    n_jobs=-1
)

grid_rf.fit(X_train_encoded, y_train)

print("Mejores parámetros:", grid_rf.best_params_)
print("Mejor AUC (CV):", grid_rf.best_score_)

modelo_rf_opt = grid_rf.best_estimator_


Mejores parámetros: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Mejor AUC (CV): 0.9906251976914414


La optimización de hiperparámetros aplicada al modelo Random Forest permitió confirmar que la configuración inicial ya ofrecía un desempeño elevado. A través de la búsqueda en grilla se identificó una combinación de parámetros que mantiene una alta capacidad de discriminación, reflejada en un valor de ROC-AUC cercano a uno en validación cruzada.
la optimización refuerza la solidez del modelo y valida su capacidad de generalización

# 10) Evaluación final y elección del mejor modelo

Se evaluaron los modelos finales utilizando métricas de clasificación adecuadas, comparando su desempeño sobre el conjunto de test.

A partir de estos resultados, se seleccionó el modelo que ofrece el mejor compromiso entre desempeño predictivo y estabilidad, justificando su elección en función de los objetivos del trabajo.

In [118]:
# Evalúa el Random Forest optimizado sobre el conjunto de prueba
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

y_pred_rf_opt = modelo_rf_opt.predict(X_test_encoded)

accuracy_rf_opt = accuracy_score(y_test, y_pred_rf_opt)
print("Accuracy Random Forest Optimizado:", accuracy_rf_opt)

print("\nReporte de clasificación Random Forest Optimizado:")
print(classification_report(y_test, y_pred_rf_opt))

print("\nMatriz de confusión Random Forest Optimizado:")
print(confusion_matrix(y_test, y_pred_rf_opt))


Accuracy Random Forest Optimizado: 0.9566974595842956

Reporte de clasificación Random Forest Optimizado:
              precision    recall  f1-score   support

           0       0.96      0.97      0.96      2915
           1       0.96      0.94      0.95      2281

    accuracy                           0.96      5196
   macro avg       0.96      0.96      0.96      5196
weighted avg       0.96      0.96      0.96      5196


Matriz de confusión Random Forest Optimizado:
[[2818   97]
 [ 128 2153]]


In [119]:
# Calcular accuracy para el modelo de Regresión Logística
# Se utilizan los datos codificados, consistentes con el entrenamiento del modelo
from sklearn.metrics import accuracy_score

# Generar predicciones sobre el conjunto de prueba
y_pred_lr = modelo_logistico.predict(X_test_encoded)

# Calcular accuracy
accuracy_lr = accuracy_score(y_test, y_pred_lr)

# Mostrar resultado
print(f"Accuracy Regresión Logística: {accuracy_lr:.4f}")


Accuracy Regresión Logística: 0.7698


In [120]:
# Crear tabla comparativa usando las métricas ya calculadas
import pandas as pd

results = pd.DataFrame({
    'Modelo': ['Regresión Logística', 'Random Forest', 'XGBoost'],
    'Accuracy': [accuracy_lr, accuracy_rf, accuracy_xgb],
    'ROC-AUC': [auc_logistico, auc_rf, roc_auc_xgb]
})

results


,Modelo,Accuracy,ROC-AUC
0,Regresión Logística,0.769823,"[0.8741987902861785, 0.8535260280310554, 0.867..."
1,Random Forest,0.956313,"[0.9895428966928471, 0.992867517340736, 0.9908..."
2,XGBoost,0.958045,0.994381


La evaluación final de los modelos permitió comparar de manera directa su desempeño sobre el conjunto de prueba, utilizando métricas consistentes con el problema de clasificación planteado. Los resultados confirman las diferencias observadas en las etapas previas de modelado y validación cruzada.

La Regresión Logística, utilizada como modelo base, presenta un desempeño aceptable, aunque limitado en su capacidad predictiva frente a modelos más complejos. Por su parte, el modelo de Random Forest optimizado muestra un rendimiento elevado y estable, evidenciando una mejora sustancial respecto al modelo base y un buen equilibrio entre precisión y capacidad de generalización.

Sin embargo, XGBoost se consolida como el modelo con mejor desempeño global, alcanzando los valores más altos tanto en accuracy como en ROC-AUC. Estos resultados indican una mayor capacidad de discriminación entre clases y una mejor generalización sobre datos no vistos. En función de este análisis, XGBoost se selecciona como el modelo final recomendado para abordar la predicción de la satisfacción de los pasajeros.

# 11) Conclusiones

El objetivo de este trabajo fue predecir la satisfacción de los pasajeros a partir de variables vinculadas tanto a aspectos operativos como a la experiencia del servicio. Para ello, se desarrolló un proceso completo que incluyó análisis exploratorio de los datos, limpieza y preparación del dataset, entrenamiento de distintos modelos de clasificación y evaluación comparativa de su desempeño.

Entre los modelos evaluados, la Regresión Logística funcionó adecuadamente como modelo base, permitiendo establecer una referencia inicial de desempeño. Sin embargo, los modelos basados en árboles mostraron una capacidad superior para capturar relaciones no lineales e interacciones complejas entre las variables. En particular, tanto Random Forest como XGBoost alcanzaron valores elevados de accuracy y otras métricas, evidenciando una muy buena capacidad de generalización.

La validación cruzada confirmó que el desempeño obtenido por los modelos es consistente y no depende de una única forma de dividir los datos. Además, el ajuste de hiperparámetros realizado sobre el modelo de Random Forest evidenció que su rendimiento es estable ante cambios en su configuración, reforzando su capacidad de generalización y su solidez predictiva.

No obstante, XGBoost se consolidó como el modelo con mejor desempeño global, alcanzando los mayores valores tanto de accuracy como de ROC-AUC. Estos resultados reflejan una mayor capacidad de discriminación entre clases y una mejor generalización sobre datos no vistos, lo que lo posiciona como la alternativa más adecuada para abordar el problema planteado.

En conclusión, XGBoost fue seleccionado como el mejor modelo para la predicción de la satisfacción de los pasajeros, debido a su alto desempeño predictivo, estabilidad y capacidad para modelar relaciones complejas en los datos. Más allá de la predicción, el enfoque desarrollado permite obtener información valiosa sobre los factores que influyen en la experiencia del cliente, aportando una base sólida para la toma de decisiones estratégicas orientadas a la mejora del servicio y la satisfacción general del pasajero.